# TODOS LOS FORMATOS

In [358]:
import pandas as pd
from datetime import datetime
import numpy as np

### Ventas y Volumen

In [359]:
cols = ['Año', 'Mes', 'Direccion', 'Punto Operacional', 'Sector', 'Seccion', 'Grupo de Familia', 'Ventas c/impuesto', 'Venta en Unidades']
df_ventas_vol = pd.read_csv('data/tercer semana - briefing/28 dias/ventas y volumen total cia tercer semana briefing.csv', encoding='utf-16', header=1, usecols=cols, decimal=',')
df_ventas_vol.columns = df_ventas_vol.columns.str.lower().str.replace(' ','_')

In [360]:
df_ventas_vol.rename(columns={
    'mes':'fecha',
    'ventas_c/impuesto':'vct',
    'venta_en_unidades':'vol',
}, inplace=True)

In [361]:
df_ventas_vol['mes'] = df_ventas_vol['fecha'].str.split(' ').str[0]
df_ventas_vol['numero_operacional'] = df_ventas_vol['punto_operacional'].str.split(' ').str[0].astype(int)

### Separacion Ventas y Volumen

In [362]:
df_ventas = df_ventas_vol[['año', 'fecha', 'direccion', 'punto_operacional', 'sector', 'seccion', 'grupo_de_familia', 'vct', 'mes', 'numero_operacional']]
df_volumen = df_ventas_vol[['año', 'fecha', 'direccion', 'punto_operacional', 'sector', 'seccion', 'grupo_de_familia', 'vol', 'mes', 'numero_operacional']]
df_ventas.rename(columns={
    'vct':'valores'
}, inplace=True)
df_volumen.rename(columns={
    'vol':'valores'
}, inplace=True)

C:\Users\juan_mera\AppData\Local\Temp\ipykernel_15932\3609712812.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ventas.rename(columns={
C:\Users\juan_mera\AppData\Local\Temp\ipykernel_15932\3609712812.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_volumen.rename(columns={


In [363]:
df_ventas['categoria'] = 'vct'
df_volumen['categoria'] = 'vol'

In [364]:
df_ventas['valores'] = pd.to_numeric(df_ventas['valores'].str.replace('.', '').str.replace(',','.'))
df_volumen['valores'] = pd.to_numeric(df_volumen['valores'].str.replace('.', '').str.replace(',','.'))

### Le quito los envases al volumen

In [365]:
df_volumen = df_volumen[~df_volumen['grupo_de_familia'].isin(['ENVASES BEBIDAS', 'ENVASES PAGADOS'])]

In [366]:
df_ventas_tienda = df_ventas.groupby(['año', 'mes', 'direccion', 'numero_operacional', 'punto_operacional', 'categoria'])['valores'].sum().reset_index()
df_volumen_tienda = df_volumen.groupby(['año', 'mes', 'direccion', 'numero_operacional', 'punto_operacional', 'categoria'])['valores'].sum().reset_index()

### Padron

In [367]:
cols = ['N°', 'NOMBRE', 'Fecha apertura', 'BANDERA', 'ORGANIZACIÓN ', 'PROVINCIA', 'FIN DE CIERRE', 'ENE.2', 'FEB.2', 'MAR.2', 'ABR.2', 'MAY.2', 'JUN.2', 'JUL.2', 'AGO.2', 'SEP.2', 'OCT.2', 'NOV.2', 'DIC.2']
padron = pd.read_excel('data/tercer semana - briefing/padron.xlsx', header=17, usecols=cols)
padron.columns = padron.columns.str.lower().str.strip().str.replace(' ', '_').str.replace('.2', '')
padron.rename(columns={
    'n°':'numero_operacional',
}, inplace=True)

In [368]:
#Elimino las filas que tengan na en su numero, nombre y mes comparable
padron = padron.dropna(subset=['numero_operacional', 'nombre', 'sep'], how='any')

In [369]:
padron['numero_operacional'] = padron['numero_operacional'].astype(int)

### Debitos total Tienda

In [370]:
df_debitos_tienda = pd.read_csv('data/tercer semana - briefing/28 dias/debitos x tienda total cia tercer semana briefing.csv', encoding='utf-16', header=1, decimal=',')
df_debitos_tienda.columns = df_debitos_tienda.columns.str.lower().str.replace(' ', '_')
df_debitos_tienda.rename(columns={
    'cant._tickets_por_local':'valores',
    'mes':'fecha',
}, inplace=True)
df_debitos_tienda['valores'] = pd.to_numeric(df_debitos_tienda['valores'].str.replace('.', '').str.replace(',','.'))
df_debitos_tienda['mes'] = df_debitos_tienda['fecha'].str.split(' ').str[0]
df_debitos_tienda['numero_operacional'] = df_debitos_tienda['punto_operacional'].str.split(' ').str[0].astype(int)
df_debitos_tienda = df_debitos_tienda.drop(columns=['indicadores', 'fecha'])
df_debitos_tienda['categoria'] = 'deb'
df_debitos_tienda = df_debitos_tienda[['año', 'mes', 'direccion', 'numero_operacional', 'punto_operacional', 'categoria', 'valores']]

### Concateno todo nivel tienda y Join con el Padron

In [371]:
df_tienda = pd.concat([df_ventas_tienda, df_debitos_tienda, df_volumen_tienda])
df_tienda['numero_operacional'] = df_tienda['numero_operacional'].astype(int)

In [372]:
df_tienda_join = pd.merge(df_tienda, padron, how='left', on='numero_operacional')
df_tienda_join = df_tienda_join[['año', 'mes', 'direccion', 'numero_operacional', 'punto_operacional', 'provincia', 'fecha_apertura', 'fin_de_cierre','categoria', 'sep', 'valores']]
df_tienda_comparable = df_tienda_join[df_tienda_join['sep'] == 'SC']

#Este df NO COMPARABLE me sirve para calcular las progresiones 28 dias moviles por formato
df_tienda_no_comparable = df_tienda_join

### Genero un df por Superficie comparable, pivoteo la info y calculo las porgresiones totales por tienda por categoria

In [373]:
df_tienda_comparable = df_tienda_comparable.pivot_table(values='valores', index=['direccion', 'numero_operacional', 'punto_operacional', 'categoria'], columns='año', aggfunc='sum').reset_index()
df_tienda_comparable['progresion'] = round((df_tienda_comparable[2025] / df_tienda_comparable[2024]) - 1, 3)
df_tienda_comparable = df_tienda_comparable.sort_values(by='progresion', ascending=False)

#Me sirve para luego la consolidacion de informacion
df_tienda_comparable_aux = df_tienda_comparable

In [374]:
df_tienda_no_comparable = df_tienda_no_comparable.pivot_table(values='valores', index=['direccion', 'numero_operacional', 'punto_operacional', 'categoria'], columns='año', aggfunc='sum').reset_index()
df_tienda_no_comparable['progresion'] = round((df_tienda_no_comparable[2025] / df_tienda_no_comparable[2024]) - 1, 3)
df_tienda_no_comparable = df_tienda_no_comparable.sort_values(by='progresion', ascending=False)

### Genero un DF por superficie comparable, agrupo la informacion a nivel formato para calular sus progresiones por categoria

In [375]:
df_formato_comparable = df_tienda_comparable.groupby(['direccion', 'categoria'])[[2024, 2025]].sum().reset_index()
df_formato_comparable['progresion'] = round(df_formato_comparable[2025] / df_formato_comparable[2024] - 1, 3)
df_formato_comparable_final = df_formato_comparable.sort_values(['categoria'])

In [376]:
df_formato_no_comparable = df_tienda_no_comparable.groupby(['direccion', 'categoria'])[[2024, 2025]].sum().reset_index()
df_formato_no_comparable['progresion'] = round(df_formato_no_comparable[2025] / df_formato_no_comparable[2024] - 1, 3)
df_formato_no_comparable_final = df_formato_no_comparable.sort_values(['categoria'])

### Debitos por Sector

In [377]:
df_debitos_sector = pd.read_csv('data/tercer semana - briefing/28 dias/debitos x sector total cia tercer semana briefing.csv', encoding='utf-16', header=1, decimal=',')

In [378]:
df_debitos_sector.columns = df_debitos_sector.columns.str.strip().str.lower().str.replace(' ', '_')
df_debitos_sector = df_debitos_sector.rename(columns={
    'cantidad_de_tickets':'valores',
    'mes':'fecha'
})
df_debitos_sector = df_debitos_sector.drop(columns=['indicadores'])
df_debitos_sector['mes'] = df_debitos_sector['fecha'].str.split(' ').str[0]
df_debitos_sector['numero_operacional'] = df_debitos_sector['punto_operacional'].str.split(' ').str[0]
df_debitos_sector['categoria'] = 'deb'
df_debitos_sector['valores'] = pd.to_numeric(df_debitos_sector['valores'].str.replace('.', '').str.replace(',','.'))

### Agrupo las tres categorias por Sector, las joineo con el padron, me quedo con los SC, calculo proresiones y pivoteo para mostrar mejor la informacion

In [379]:
df_ventas_sector = df_ventas.groupby(['año', 'mes', 'direccion', 'numero_operacional', 'punto_operacional', 'categoria', 'sector'])['valores'].sum().reset_index()
df_volumen_sector = df_volumen.groupby(['año', 'mes', 'direccion', 'numero_operacional', 'punto_operacional', 'categoria', 'sector'])['valores'].sum().reset_index()
df_debitos_sector = df_debitos_sector[['año', 'mes', 'direccion', 'numero_operacional', 'punto_operacional', 'categoria', 'sector', 'valores']]

In [380]:
df_sector = pd.concat([df_ventas_sector, df_volumen_sector, df_debitos_sector])
df_sector['numero_operacional'] = df_sector['numero_operacional'].astype(int) 
df_sector_join = pd.merge(df_sector, padron, on='numero_operacional', how='left')
df_sector_join = df_sector_join[['año', 'mes', 'direccion', 'numero_operacional', 'punto_operacional', 'provincia', 'fecha_apertura', 'fin_de_cierre', 'categoria', 'sector', 'sep', 'valores']]

In [381]:
df_sector_comparable = df_sector_join[df_sector_join['sep'] == 'SC']

In [382]:
df_sector_comparable = df_sector_comparable.pivot_table(values='valores', index=['direccion' ,'numero_operacional', 'punto_operacional', 'categoria', 'sector'], columns='año', aggfunc='sum').reset_index()
df_sector_comparable['progresion'] = round((df_sector_comparable[2025] / df_sector_comparable[2024]) - 1, 3)

In [383]:
#Sirve para calcular las progresiones por sector a nivel Formato
df_formato_sector_comparable = df_sector_comparable

#Sirve Para realizar una baja consolidada de informacion
df_formato_sector_comparable_aux = df_formato_sector_comparable

In [384]:
df_progresiones_categoria_sectores = df_sector_comparable.pivot_table(values='progresion', index=['numero_operacional', 'punto_operacional', 'categoria'], columns='sector', aggfunc='sum').reset_index()

In [385]:
df_tienda_comparable = df_tienda_comparable.rename(columns={'progresion':'total_tienda'})

### Trabajo con la Info para Obtener las progresiones por Sector a Nivel Formato

In [386]:
df_formato_sector_comparable = df_formato_sector_comparable.groupby(['direccion', 'categoria', 'sector'])[[2024, 2025]].sum().reset_index()

In [387]:
df_formato_sector_comparable['progresion'] = round(df_formato_sector_comparable[2025] / df_formato_sector_comparable[2024] - 1, 3)

In [388]:
df_formato_sector_comparable = df_formato_sector_comparable.pivot_table(values='progresion', index=['direccion', 'categoria'], columns='sector', aggfunc='sum').reset_index()
df_formato_sector_comparable = df_formato_sector_comparable.fillna(0)

### Concateno las progresiones por sectores con el primer df utilziando su numero operacional. Luego lo limpio y lo presento

In [389]:
df_progresiones_join_sector_tienda = pd.merge(df_progresiones_categoria_sectores, df_tienda_comparable[['direccion', 'numero_operacional', 'categoria', 'total_tienda']], on=['numero_operacional', 'categoria'], how='left')

In [390]:
df_progresiones_join_sector_tienda = df_progresiones_join_sector_tienda.fillna(0)

In [391]:
df_progresiones_join_sector_tienda.columns = df_progresiones_join_sector_tienda.columns.str.capitalize().str.strip().str.replace('_', ' ')

In [392]:
df_progresiones_join_sector_tienda = df_progresiones_join_sector_tienda.drop(columns=['Numero operacional'])
df_progresiones_join_sector_tienda = df_progresiones_join_sector_tienda.rename(columns={'Total tienda': 'Total tienda', 'P.g.c.': 'PGC'})
df_progresiones_join_sector_tienda = df_progresiones_join_sector_tienda.sort_values(by='Total tienda', ascending=False)

In [393]:
df_final_consolidado_tienda = df_tienda_comparable.drop(columns=[2024, 2025])
df_final_consolidado_tienda = df_final_consolidado_tienda.rename(columns={'total_tienda':'progresion'})
df_final_consolidado_tienda['sector'] = 'Total'

In [394]:
df_final_consolidado_sector = df_sector_comparable.drop(columns=[2024, 2025])
df_final_consolidado_sector[['direccion', 'numero_operacional', 'punto_operacional', 'categoria', 'progresion', 'sector']]

año,direccion,numero_operacional,punto_operacional,categoria,progresion,sector
0,E-COMMERCE,899,899 - E-Commerce,deb,-0.586,BAZAR
1,E-COMMERCE,899,899 - E-Commerce,deb,-0.057,ELECTRODOMESTICOS
2,E-COMMERCE,899,899 - E-Commerce,deb,2.000,P.G.C.
3,E-COMMERCE,899,899 - E-Commerce,deb,NaN,TEXTIL
4,E-COMMERCE,899,899 - E-Commerce,vct,-0.740,BAZAR
...,...,...,...,...,...,...
11613,PROXIMIDAD,874,874 - Mini Caraffa Claudio Cuenca 1784 Cb,vol,-0.750,ELECTRODOMESTICOS
11614,PROXIMIDAD,874,874 - Mini Caraffa Claudio Cuenca 1784 Cb,vol,0.306,OTROS
11615,PROXIMIDAD,874,874 - Mini Caraffa Claudio Cuenca 1784 Cb,vol,0.317,P.G.C.
11616,PROXIMIDAD,874,874 - Mini Caraffa Claudio Cuenca 1784 Cb,vol,0.522,PRODUCTOS FRESCO


In [395]:
df_final_consolidado_total = pd.concat([df_final_consolidado_sector, df_final_consolidado_tienda])
df_final_consolidado_total = df_final_consolidado_total.pivot_table(values='progresion', index=['direccion', 'punto_operacional'], columns=['categoria', 'sector'], aggfunc='sum').reset_index()

### Trabajo sobre el ultimo punto "Grupo de Familia". Agrupo, joineo con el padron, me quedo con los SC, calculo el gap y la CMG, limpio la info y presento

In [396]:
df_volumen_grupo_de_familia = df_volumen.groupby(['año', 'mes', 'direccion', 'numero_operacional', 'punto_operacional', 'categoria', 'sector', 'seccion', 'grupo_de_familia'])['valores'].sum().reset_index()

In [397]:
df_volumen_grupo_de_familia_join = pd.merge(df_volumen_grupo_de_familia, padron[['numero_operacional', 'sep']], on='numero_operacional', how='left')

In [398]:
df_volumen_grupo_de_familia_comparable = df_volumen_grupo_de_familia_join[df_volumen_grupo_de_familia_join['sep'] == 'SC']

In [399]:
df_volumen_grupo_de_familia_comparable = df_volumen_grupo_de_familia_comparable.pivot_table(values='valores', index=['direccion', 'grupo_de_familia', 'seccion', 'categoria'], columns='año', aggfunc='sum').reset_index()

In [400]:
df_volumen_grupo_de_familia_comparable[2024] = df_volumen_grupo_de_familia_comparable[2024].fillna(0)

In [401]:
df_volumen_grupo_de_familia_comparable['GAP'] = df_volumen_grupo_de_familia_comparable[2025] - df_volumen_grupo_de_familia_comparable[2024]

In [402]:
df_volumen_grupo_de_familia_comparable['progresion'] = (df_volumen_grupo_de_familia_comparable[2025] / df_volumen_grupo_de_familia_comparable[2024]) - 1

In [403]:
df_volumen_grupo_de_familia_comparable['progresion'] = df_volumen_grupo_de_familia_comparable['progresion'].replace(np.inf, 0)
df_volumen_grupo_de_familia_comparable['progresion'] = df_volumen_grupo_de_familia_comparable['progresion'].replace(np.nan, 0)
df_volumen_grupo_de_familia_comparable.sort_values('progresion', ascending=False)

año,direccion,grupo_de_familia,seccion,categoria,2024,2025,GAP,progresion
132,HIPERMERCADO,FOTOGRAFIA,FOTOGRAFIA,vol,1.00,332.00,331.00,331.000000
410,MARKET,MERCADO DEL MAR CONGELADO,PESCADERIA,vol,5.47,1164.55,1159.08,211.897623
291,MARKET,BAGAJE,LIBRERIA,vol,6.00,395.00,389.00,64.833333
304,MARKET,CALZADO AGUA VERANO,CALZADO,vol,1.00,64.00,63.00,63.000000
699,No informado,CONSUMIBLES ELECTRONICA,SONIDO,vol,1.00,36.00,35.00,35.000000
...,...,...,...,...,...,...,...,...
429,MARKET,PLANTINES,FRUTAS Y VERDURAS,vol,1595.00,13.00,-1582.00,-0.991850
841,PROXIMIDAD,ROPA DE BANO PERMANENTE,BLANCO,vol,201.00,1.00,-200.00,-0.995025
463,MARKET,TARJETA GIFT CARD,OTRAS VENTAS,vol,-1.00,0.00,1.00,-1.000000
676,MAXI,TARJETA GIFT CARD,OTRAS VENTAS,vol,-1.00,0.00,1.00,-1.000000


In [404]:
df_volumen_grupo_de_familia_comparable['total_2024_direccion'] = df_volumen_grupo_de_familia_comparable.groupby('direccion')[2024].transform('sum')
df_volumen_grupo_de_familia_comparable['Cmg'] = df_volumen_grupo_de_familia_comparable['GAP'] / df_volumen_grupo_de_familia_comparable['total_2024_direccion']

In [405]:
df_volumen_grupo_de_familia_comparable['Cmg'] = df_volumen_grupo_de_familia_comparable['Cmg'].fillna(0)
df_volumen_grupo_de_familia_comparable = df_volumen_grupo_de_familia_comparable.sort_values('Cmg', ascending=False)
df_volumen_grupo_de_familia_comparable = df_volumen_grupo_de_familia_comparable.rename(columns={'direccion':'Direccion','grupo_de_familia':'Grupo de familia', 'seccion':'Seccion', 'categoria':'Categoria', 'progresion':'Progresion'})
df_volumen_grupo_de_familia_comparable = df_volumen_grupo_de_familia_comparable.drop(columns=['total_2024_direccion'])

### Trabajo sobre una bajada de informacion consolidada para realizar un giratorio

In [406]:
# Genero un DF auxiliar para realizar una bajada consolidada de informacion para generar un giratorio
df_tienda_comparable_aux['sector'] = ''
df_tienda_comparable_aux = df_tienda_comparable_aux[['direccion', 'numero_operacional', 'punto_operacional', 'categoria', 'sector', 2024, 2025, 'progresion']]
df_tienda_comparable_aux['aux'] = 'tienda'

df_formato_comparable_aux = df_formato_comparable.copy()
df_formato_comparable_aux['sector'] = ''
df_formato_comparable_aux['numero_operacional'] = ''
df_formato_comparable_aux['punto_operacional'] = ''
df_formato_comparable_aux = df_formato_comparable_aux[['direccion', 'numero_operacional', 'punto_operacional', 'categoria', 'sector', 2024, 2025, 'progresion']]
df_formato_comparable_aux['aux'] = 'formato'

df_formato_sector_comparable_aux['aux'] = 'sector'


# ME FALTA AGREGAR UNA TAB QUE SEA TOTAL FORMATO APERTURADO POR SECTOR PARA REALIZAR EL GIRATORIO
df_bajada_consolidada = pd.concat([df_tienda_comparable_aux, df_formato_comparable_aux, df_formato_sector_comparable_aux])

# GRAFICOS V2

In [407]:
deb_acum = pd.read_csv('data/tercer semana - briefing/acum/debitos x tienda total cia acum 2023 - 2025.csv', encoding='utf-16', header=1, decimal=',')
cols = ['Año', 'Mes', 'Direccion', 'Punto Operacional', 'Ventas c/impuesto']
vct_acum = pd.read_csv('data/tercer semana - briefing/acum/ventas y volumen total cia acum 2023 - 2025.csv', encoding='utf-16', header=1, usecols=cols)
vol_acum = pd.read_csv('data/tercer semana - briefing/acum/volumen sin envase total cia acum 2023 - 2025.csv', encoding='utf-16', header=1)

In [408]:
deb_acum = deb_acum.drop(columns=['Indicadores'])
vol_acum = vol_acum.drop(columns=['Indicadores'])

In [409]:
deb_acum = deb_acum.rename(columns={'Cant. Tickets por Local':'valores'})
vol_acum = vol_acum.rename(columns={'VOLUMEN':'valores'})
vct_acum = vct_acum.rename(columns={'Ventas c/impuesto':'valores'})

In [410]:
deb_acum['categoria'] = 'deb'
vol_acum['categoria'] = 'vol'
vct_acum['categoria'] = 'vct'

In [411]:
vol_acum = vol_acum[['Año', 'Mes', 'Direccion', 'Punto Operacional', 'valores', 'categoria']]

In [412]:
vol_acum['valores'] = pd.to_numeric(vol_acum['valores'].str.replace('.','').str.replace(',', '.'))
vct_acum['valores'] = pd.to_numeric(vct_acum['valores'].str.replace('.','').str.replace(',', '.'))
deb_acum['valores'] = pd.to_numeric(deb_acum['valores'].str.replace('.', ''))

In [413]:
acum_join = pd.concat([deb_acum, vol_acum, vct_acum])

In [414]:
acum_join.columns = acum_join.columns.str.strip().str.replace(' ', '_').str.lower()

In [415]:
acum_join['numero_operacional'] = acum_join['punto_operacional'].str.split(' ').str[0].astype(int)

In [416]:
acum_join = pd.merge(acum_join, padron[['numero_operacional', 'sep']], how='left')

In [417]:
acum_join_comparable = acum_join[acum_join['sep'] == 'SC']
acum_join_comparable = acum_join_comparable.rename(columns={'mes':'fecha'})
acum_join_comparable['mes'] = acum_join_comparable['fecha'].str.split(' ').str[0]

acum_join_no_comparable = acum_join
acum_join_no_comparable = acum_join_no_comparable.rename(columns={'mes':'fecha'})
acum_join_no_comparable['mes'] = acum_join_no_comparable['fecha'].str.split(' ').str[0] 

In [420]:
acum_join_comparable = acum_join_comparable.groupby(['año', 'fecha', 'mes', 'direccion', 'categoria'])['valores'].sum().reset_index()

acum_join_no_comparable = acum_join_no_comparable.groupby(['año', 'fecha', 'mes', 'direccion', 'categoria'])['valores'].sum().reset_index()

In [421]:
meses_orden = {'Enero':1, 'Febrero':2, 'Marzo':3, 'Abril':4, 'Mayo':5, 'Junio':6, 'Julio':7, 'Agosto':8, 'Septiembre':9, 'Octubre':10, 'Noviembre':11, 'Diciembre':12}

In [422]:
acum_join_comparable['aux'] = acum_join_comparable['mes'].map(meses_orden)

acum_join_no_comparable['aux'] = acum_join_no_comparable['mes'].map(meses_orden)

In [423]:
meses_orden.items()
meses_invertidos = {v:k for k, v in meses_orden.items()}
meses_invertidos

{1: 'Enero',
 2: 'Febrero',
 3: 'Marzo',
 4: 'Abril',
 5: 'Mayo',
 6: 'Junio',
 7: 'Julio',
 8: 'Agosto',
 9: 'Septiembre',
 10: 'Octubre',
 11: 'Noviembre',
 12: 'Diciembre'}

In [424]:
acum_join_comparable.sort_values(by='aux', ascending=True)
acum_join_comparable['fecha_completa'] = pd.to_datetime(
    '01/' + acum_join_comparable['aux'].astype(str) + '/' + acum_join_comparable['año'].astype(str), format='%d/%m/%Y')
mes_comparable = 'Septiembre'
fecha_tope = pd.to_datetime('01/'+ str(meses_orden[mes_comparable]) + '/' + '2025', format='%d/%m/%Y')


acum_join_no_comparable.sort_values(by='aux', ascending=True)
acum_join_no_comparable['fecha_completa'] = pd.to_datetime(
    '01/' + acum_join_no_comparable['aux'].astype(str) + '/' + acum_join_no_comparable['año'].astype(str), format='%d/%m/%Y')
mes_comparable = 'Septiembre'
fecha_tope = pd.to_datetime('01/'+ str(meses_orden[mes_comparable]) + '/' + '2025', format='%d/%m/%Y')

In [425]:
acum_join_comparable = acum_join_comparable[acum_join_comparable['fecha_completa'] <= fecha_tope]

acum_join_no_comparable = acum_join_no_comparable[acum_join_no_comparable['fecha_completa'] <= fecha_tope]

In [426]:
acum_join_comparable = acum_join_comparable.pivot_table(values='valores', columns='año', index=['direccion', 'mes', 'categoria', 'aux'], aggfunc='sum').reset_index()

acum_join_no_comparable = acum_join_no_comparable.pivot_table(values='valores', columns='año', index=['direccion', 'mes', 'categoria', 'aux'], aggfunc='sum').reset_index()

In [427]:
acum_join_comparable['progresion 2024'] = round((acum_join_comparable[2024] / acum_join_comparable[2023]) - 1, 3)
acum_join_comparable['progresion 2025'] = round((acum_join_comparable[2025] / acum_join_comparable[2024]) - 1, 3)

acum_join_no_comparable['progresion 2024'] = round((acum_join_no_comparable[2024] / acum_join_no_comparable[2023]) - 1, 3)
acum_join_no_comparable['progresion 2025'] = round((acum_join_no_comparable[2025] / acum_join_no_comparable[2024]) - 1, 3)

In [428]:
acum_join_comparable = acum_join_comparable.sort_values(by=['direccion', 'categoria' ,'aux'], ascending=[True, True, True])
acum_join_comparable = acum_join_comparable.drop(columns=['aux'])

acum_join_no_comparable = acum_join_no_comparable.sort_values(by=['direccion', 'categoria' ,'aux'], ascending=[True, True, True])
acum_join_no_comparable = acum_join_no_comparable.drop(columns=['aux'])

### Genero dos bucles para poder realizar varios filtros en las tablas importantes y asi conseguir un archivo por formato donde se muestre la informacion Total a Nivel Formato, Tienda y Sector

In [429]:
from pathlib import Path

formatos = df_tienda_comparable['direccion'].unique().tolist()
categorias = ['vct', 'deb', 'vol']

try:
    out_dir = Path("results")
    out_dir.mkdir(parents=True, exist_ok=True)

    for formato in formatos:
        out_file = out_dir / f"Resultados Briefing {formato.upper()} ({datetime.today().strftime('%d-%m-%Y')}).xlsx"

        with pd.ExcelWriter(out_file) as writer:

            df_formato_comparable_final[df_formato_comparable_final['direccion'] == formato].to_excel(writer, sheet_name=f'Total Categoria - {formato[0:3]}', index=False)

            df_formato_no_comparable_final[df_formato_no_comparable_final['direccion'] == formato].to_excel(writer, sheet_name=f'Total Categoria (Sup Total) - {formato[0:3]}'[0:31], index=False)

            df_formato_sector_comparable[df_formato_sector_comparable['direccion'] == formato].to_excel(writer, sheet_name=f'Total Categoria x Sector - {formato[0:3]}', index=False)
            
            for categoria in categorias:
                df_progresiones_join_sector_tienda[(df_progresiones_join_sector_tienda['Categoria'] == categoria) & (df_progresiones_join_sector_tienda['Direccion'] == formato)].to_excel(writer, sheet_name=f'{categoria} - {formato[0:3]}', index=False)

            df_final_consolidado_total[df_final_consolidado_total['direccion'] == formato].to_excel(writer, sheet_name=f'Info Consolidada - {formato[0:3]}', index=True)
            
            df_volumen_grupo_de_familia_comparable[df_volumen_grupo_de_familia_comparable['Direccion'] == formato].to_excel(writer, sheet_name=f'GF Consolidada - {formato[0:3]}', index=False)

            acum_join_comparable[acum_join_comparable['direccion'] == formato].to_excel(writer, sheet_name=f'Progresiones comp - {formato[0:3]}', index=False)

            acum_join_no_comparable[acum_join_no_comparable['direccion'] == formato].to_excel(writer, sheet_name=f'Progresiones total - {formato[0:3]}', index=False)

            #Por el momento la quito ya que primero tengo que pensar como realizar el giratorio
            #df_bajada_consolidada.to_excel(writer, sheet_name=f'Base Giratorio', index=False)
            
        print(f"Archivo guardado en: {out_file}")

except Exception as e:
    print(f"No se logró guardar la información en un archivo Excel. ERROR: {e}")

Archivo guardado en: results\Resultados Briefing NO INFORMADO (21-10-2025).xlsx
Archivo guardado en: results\Resultados Briefing PROXIMIDAD (21-10-2025).xlsx
Archivo guardado en: results\Resultados Briefing MARKET (21-10-2025).xlsx
Archivo guardado en: results\Resultados Briefing MAXI (21-10-2025).xlsx
Archivo guardado en: results\Resultados Briefing HIPERMERCADO (21-10-2025).xlsx
Archivo guardado en: results\Resultados Briefing E-COMMERCE (21-10-2025).xlsx


### Graficos

In [ ]:
df_debitos = pd.read_csv('data/debitos maxi 2023 - 2025.csv', encoding='utf-16', header=1, decimal=',')

FileNotFoundError: [Errno 2] No such file or directory: 'data/debitos maxi 2023 - 2025.csv'

In [ ]:
meses_orden = {'Enero':1, 'Febrero':2, 'Marzo':3, 'Abril':4, 'Mayo':5, 'Junio':6, 'Julio':7, 'Agosto':8, 'Septiembre':9, 'Octubre':10, 'Noviembre':11, 'Diciembre':12}

In [ ]:
df_debitos.columns = df_debitos.columns.str.strip().str.replace(' ', '_').str.lower()
df_debitos = df_debitos.rename(columns={'mes':'fecha', 'cant._tickets_por_local': 'valores'})
df_debitos['mes'] = df_debitos['fecha'].str.split(' ').str[0]
df_debitos['numero_operacional'] = df_debitos['punto_operacional'].str.split(' ').str[0].astype(int)
df_debitos = df_debitos.drop(columns=['indicadores'])
df_debitos['valores'] = df_debitos['valores'].str.replace('.','').str.replace(',','.').astype(int)
df_debitos['categoria'] = 'deb'

In [ ]:
df_debitos

,año,fecha,punto_operacional,direccion,valores,mes,numero_operacional,categoria
0,2023,Enero del 2023,020 - Avellaneda,MAXI,40016,Enero,20,deb
1,2023,Enero del 2023,040 - Temperley,MAXI,34536,Enero,40,deb
2,2023,Enero del 2023,045 - Trelew 25 de Mayo,MAXI,30130,Enero,45,deb
3,2023,Enero del 2023,057 - Tandil Villa Italia,MAXI,36672,Enero,57,deb
4,2023,Enero del 2023,059 - Mendoza Godoy Cruz,MAXI,45809,Enero,59,deb
...,...,...,...,...,...,...,...,...
1032,2025,Octubre del 2025,706 - Maxi Lomas,MAXI,5188,Octubre,706,deb
1033,2025,Octubre del 2025,707 - Maxi Cordoba J.B.Justo,MAXI,7559,Octubre,707,deb
1034,2025,Octubre del 2025,708 - Maxi Tortuguitas,MAXI,4735,Octubre,708,deb
1035,2025,Octubre del 2025,709 - MAXI - LOMA HERMOSA,MAXI,4149,Octubre,709,deb


In [ ]:
cols = ['Año', 'Mes', 'Direccion', 'Punto Operacional', 'Grupo de Familia', 'Ventas c/impuesto', 'Venta en Unidades']
df_ventas = pd.read_csv('data/ventas maxi 2023 -2025.csv', encoding='utf-16', header=1, usecols=cols)

In [ ]:
df_ventas.columns = df_ventas.columns.str.lower().str.strip().str.replace(' ', '_')

df_ventas = df_ventas.rename(columns={
    'mes':'fecha',
    'ventas_c/impuesto':'ventas',
    'venta_en_unidades':'volumen'
})

df_ventas['mes'] = df_ventas['fecha'].str.split(' ').str[0]
df_ventas['numero_operacional'] = df_ventas['punto_operacional'].str.split(' ').str[0].astype(int)

df_ventas['ventas'] = df_ventas['ventas'].str.replace('.', '').str.replace(',', '.').astype(float)
df_ventas['volumen'] = df_ventas['volumen'].str.replace('.', '').str.replace(',', '.').astype(float)

In [ ]:
df_volumen = df_ventas[['año', 'fecha', 'mes', 'direccion', 'numero_operacional', 'punto_operacional', 'grupo_de_familia', 'volumen']]
df_ventas = df_ventas[['año', 'fecha', 'mes', 'direccion', 'numero_operacional', 'punto_operacional', 'grupo_de_familia', 'ventas' ]]

In [ ]:
df_ventas = df_ventas.rename(columns={
    'ventas':'valores'
})
df_ventas['categoria'] =  'vct'
df_ventas = df_ventas.groupby(['año', 'fecha', 'mes', 'direccion', 'numero_operacional', 'punto_operacional', 'categoria'])['valores'].sum().reset_index()

In [ ]:
df_volumen = df_volumen.rename(columns={
    'volumen':'valores'
})
df_volumen['categoria'] =  'vol'
df_volumen = df_volumen[~df_volumen['grupo_de_familia'].isin(['ENVASES BEBIDAS', 'ENVASES PAGADOS'])]
df_volumen = df_volumen.groupby(['año', 'fecha', 'mes', 'direccion', 'numero_operacional', 'punto_operacional', 'categoria'])['valores'].sum().reset_index()

In [ ]:
df_debitos = df_debitos[['año', 'fecha', 'mes', 'direccion', 'numero_operacional', 'punto_operacional', 'categoria', 'valores']]

In [ ]:
df_vdv = pd.concat([df_ventas, df_volumen, df_debitos])

In [ ]:
df_vdv_join = pd.merge(df_vdv, padron[['numero_operacional', 'sep']], on='numero_operacional')

In [ ]:
df_vdv_join['aux'] = df_vdv_join['mes'].map(meses_orden)

In [ ]:
meses_orden.items()
meses_invertidos = {v:k for k, v in meses_orden.items()}
meses_invertidos

{1: 'Enero',
 2: 'Febrero',
 3: 'Marzo',
 4: 'Abril',
 5: 'Mayo',
 6: 'Junio',
 7: 'Julio',
 8: 'Agosto',
 9: 'Septiembre',
 10: 'Octubre',
 11: 'Noviembre',
 12: 'Diciembre'}

In [ ]:
df_vdv_join

,año,fecha,mes,direccion,numero_operacional,punto_operacional,categoria,valores,sep,aux
0,2023,Abril del 2023,Abril,MAXI,20,020 - Avellaneda,vct,9.440494e+08,SC,4
1,2023,Abril del 2023,Abril,MAXI,40,040 - Temperley,vct,2.997863e+08,SC,4
2,2023,Abril del 2023,Abril,MAXI,45,045 - Trelew 25 de Mayo,vct,1.980232e+08,SC,4
3,2023,Abril del 2023,Abril,MAXI,57,057 - Tandil Villa Italia,vct,3.037086e+08,SC,4
4,2023,Abril del 2023,Abril,MAXI,59,059 - Mendoza Godoy Cruz,vct,7.150335e+08,SC,4
...,...,...,...,...,...,...,...,...,...,...
3108,2025,Octubre del 2025,Octubre,MAXI,706,706 - Maxi Lomas,deb,5.188000e+03,SC,10
3109,2025,Octubre del 2025,Octubre,MAXI,707,707 - Maxi Cordoba J.B.Justo,deb,7.559000e+03,SC,10
3110,2025,Octubre del 2025,Octubre,MAXI,708,708 - Maxi Tortuguitas,deb,4.735000e+03,SC,10
3111,2025,Octubre del 2025,Octubre,MAXI,709,709 - MAXI - LOMA HERMOSA,deb,4.149000e+03,SC,10


In [ ]:
df_vdv_join.sort_values(by='aux', ascending=True)
df_vdv_join['fecha_completa'] = pd.to_datetime(
    '01/' + df_vdv_join['aux'].astype(str) + '/' + df_vdv_join['año'].astype(str), format='%d/%m/%Y')
mes_comparable = 'Septiembre'
fecha_tope = pd.to_datetime('01/'+ str(meses_orden[mes_comparable]) + '/' + '2025', format='%d/%m/%Y')
df_vdv_join = df_vdv_join[df_vdv_join['fecha_completa'] <= fecha_tope]

,año,fecha,mes,direccion,numero_operacional,punto_operacional,categoria,valores,sep,aux
90,2023,Enero del 2023,Enero,MAXI,266,266 - Parana Don Bosco,vct,7.080923e+07,SC,1
91,2023,Enero del 2023,Enero,MAXI,278,278 - MAXI Marquez,vct,1.686451e+08,SC,1
92,2023,Enero del 2023,Enero,MAXI,294,294 - Ituzaingo Blas Parera,vct,2.159977e+08,SC,1
93,2023,Enero del 2023,Enero,MAXI,701,701 - Ruta III,vct,7.070009e+08,SC,1
94,2023,Enero del 2023,Enero,MAXI,702,702 - Claypole,vct,7.535027e+08,SC,1
...,...,...,...,...,...,...,...,...,...,...
77,2023,Diciembre del 2023,Diciembre,MAXI,709,709 - MAXI - LOMA HERMOSA,vct,7.033901e+08,SC,12
78,2023,Diciembre del 2023,Diciembre,MAXI,896,896 - MAXI - SIDE STORE,vct,1.163335e+08,SC,12
2366,2023,Diciembre del 2023,Diciembre,MAXI,503,503 - Maxi San Vicente,deb,2.748600e+04,SC,12
2367,2023,Diciembre del 2023,Diciembre,MAXI,504,504 - Maxi Cacheuta Cba,deb,2.684500e+04,SC,12


In [ ]:
df_vdv_join = df_vdv_join.pivot_table(values='valores', columns='año', index=['mes', 'numero_operacional', 'punto_operacional', 'categoria', 'sep', 'aux'], aggfunc='sum').reset_index()
df_vdv_join['progresion 2024'] = round((df_vdv_join[2024] / df_vdv_join[2023]) - 1, 3)
df_vdv_join['progresion 2025'] = round((df_vdv_join[2025] / df_vdv_join[2024]) - 1, 3)
df_vdv_join_tienda = df_vdv_join.groupby(['mes', 'numero_operacional', 'punto_operacional', 'categoria', 'sep', 'aux'])[[2023, 2024, 2025, 'progresion 2024', 'progresion 2025']].sum().reset_index()
df_vdv_join_tienda = df_vdv_join_tienda.pivot_table(values=[2024, 2025, 'progresion 2024', 'progresion 2025'], columns='categoria', index=['mes', 'numero_operacional', 'punto_operacional', 'sep', 'aux'], aggfunc='sum').reset_index().sort_values(by=('aux',''))
df_vdv_join_comparable = df_vdv_join[df_vdv_join['sep'] == 'SC']
df_vdv_join_total = df_vdv_join
info_comparable = df_vdv_join_comparable.groupby(['mes', 'categoria', 'aux'])[['progresion 2024', 'progresion 2025']].sum().reset_index().pivot_table(values=['progresion 2024', 'progresion 2025'], index=['mes', 'aux'], columns='categoria', aggfunc='sum').reset_index().sort_values(('aux',''))

In [ ]:
info_comparable

año               mes aux progresion 2024                progresion 2025  \
categoria                             deb     vct    vol             deb   
3               Enero   1           1.381  60.487  0.355          -6.497   
4             Febrero   2          -0.154  70.576  0.524          -6.019   
7               Marzo   3          -2.231  61.121 -1.550          -5.124   
0               Abril   4          -4.103  51.952 -3.600          -4.165   
8                Mayo   5          -3.031  49.917 -3.630          -5.305   
6               Junio   6          -3.594  46.074 -3.974          -6.144   
5               Julio   7          -4.056  41.593 -5.538          -5.370   
1              Agosto   8          -5.862  36.663 -6.107          -4.423   
11         Septiembre   9          -6.925  46.838 -5.847          -5.486   
10            Octubre  10          -6.188  37.826 -5.516           0.000   
9           Noviembre  11           8.905  62.199  0.320           0.000   
2           Diciembre  12          -8.680  23.551 -7.878           0.000   

año                       
categoria     vct    vol  
3          20.558 -2.907  
4          18.080 -0.899  
7          15.646 -0.998  
0          16.831  0.252  
8          20.095  0.853  
6          15.307 -1.293  
5          12.593 -0.202  
1          19.019  2.991  
11          6.457 -1.154  
10          0.000  0.000  
9           0.000  0.000  
2           0.000  0.000